In [ ]:

class Callback():
    def begin_fit(self,learn):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self,epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True 
    def begin_batch(self,xb,yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self,loss):
        self.loss=loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False